In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
authors = pd.read_csv("../input/nips-2015-papers/Authors.csv")
paper_authors = pd.read_csv("../input/nips-2015-papers/PaperAuthors.csv")
papers = pd.read_csv("../input/nips-2015-papers/Papers.csv")

**Sample view on each dataframes**

In [ ]:
authors.head(5)

In [ ]:
paper_authors.head(5)

In [ ]:
papers.describe()

In [ ]:
papers.head(5)

**HIGH LEVEL APPROACH**
1. > Text PreProcessing
       a.Noise Removal
       b.Normalisation
2. > Data Exploration
       a.Word cloud to understand frequently used words
       b.Top 20 unigrams,bi-grams,tri-grams
3. > Convert text to a vector of word counts  
4. > Covert text to a vector of term frequencies
5. > Sort terms based on term frequencies in descending order to identify top N keywords

> Before preprocessing,it is advisable to explore dataset in terms of word count,most common and uncommon words 

In [ ]:
papers.isnull().describe()

> Dropping Event type column as it has no relevance at all.

In [ ]:
papers.drop(['EventType'],axis=1,inplace=True)
papers

> Words of each abstract are counted

In [ ]:
papers['word_count']=papers['Abstract'].apply(lambda x:len(str(x).split(" ")))
papers[['Abstract','word_count']].head()

In [ ]:
papers.word_count.describe()

> It indicate that average count for each word is 149 words per abstract.Min words per abstract is 58 and maximum words are 296 words.The word count is important to give us an indication of the size of the dataset that we are handling as well as the variation in word counts across the rows

> **Frequently used words**

In [ ]:
freq = pd.Series(' '.join(papers['Abstract']).split()).value_counts()[:20]
freq

> **Uncommon Words**

In [ ]:
non_freq = pd.Series(' '.join(papers['Abstract']).split()).value_counts()[-20:]
non_freq

> **Text Pre-Processing **
> Objectives
1. Text Clean Up
2. Shrinking the vocabulary to retain only relevant/important words
3. Reduce sparsity

> Noise Removal can be done by removing redundant text components like Punctuation,Tags,URLs
> Normalization can be done by Lemmatization which works based on the root of the world and Stemming that removes suffixes

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
stem = PorterStemmer()
word = "good"
print("stemming:",stem.stem(word))
print("lemmatization:", lem.lemmatize(word, "v"))

> The stemming and lemmatizaton of most of the nouns are same as that of words itself.


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

> Stop words include the large number of prepositions, pronouns, conjunctions etc in sentences. These words need to be removed before we analyse the text, so that the frequently used words are mainly the words relevant to the context and not common words used in the text.

> There is a default list of stopwords in python nltk library. In addition, we might want to add context specific stopwords for which the “most common words” that we listed in the beginning will be helpful.

In [ ]:
##Creating a list of stopwords and adding a custom list of stopwords
stop_words=set(stopwords.words('english'))
print(stop_words)

In [ ]:
##Creating a list of custom stopwords
new_words= ["using", "show", "result", "large", "also",
            "iv", "one", "two", "new", "previously", "shown"]
stop_words=stop_words.union(new_words)

**Preprocessing Text**

In [ ]:
corpus=[]
for i in range(0,403):
    #remove punctuations
    text=re.sub('^[a-zA-Z]',' ',papers.Abstract[i])
    #convert to lower case
    text=text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    #remove special characters and digits
    text=re.sub('(\\d|\\W)',' ',text)
    #Convert to list from string
    list=text.split()
    #Stemming
    ps=PorterStemmer()
    #Lemmatization
    lem=WordNetLemmatizer()
    temp=[lem.lemmatize(word) for word in list if word not in stop_words]
    text=" ".join(temp)
    corpus.append(text)
    

**Data Visualisation**

In [ ]:
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
wordcloud = WordCloud(background_color='white',stopwords=stop_words,
                                        max_words=100,
                                        max_font_size=50, 
                                        random_state=42 #near to std_dev
                                        ).generate(str(corpus))
print(wordcloud)
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig("word1.png", dpi=900)

> Tokenisation is the process of converting the continuous text into a list of words. The list of words is then converted to a matrix of integers by the process of vectorisation. Vectorisation is also called feature extraction.

**CountVectoriser to tokenise the text and build a vocabulary of known words.**
>     cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
    max_df — When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). This is to ensure that we only have words relevant to the context and not commonly used words.
    max_features — determines the number of columns in the matrix.
    n-gram range — we would want to look at a list of single words, two words (bi-grams) and three words (tri-gram) combinations.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re
cv=CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X=cv.fit_transform(corpus)


In [ ]:
#Most frequently occuring words
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in      
                   vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                       reverse=True)
    return words_freq[:n]#Convert most freq words to dataframe for plotting bar plot
top_words = get_top_n_words(corpus, n=20)
top_df = pd.DataFrame(top_words)
top_df.columns=["Word", "Freq"]#Barplot of most freq words
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
g = sns.barplot(x="Word", y="Freq", data=top_df)
g.set_xticklabels(g.get_xticklabels(), rotation=30)

In [ ]:
#Most frequently occuring Bi-grams
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(2,2),  
            max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]
top2_words = get_top_n2_words(corpus, n=20)
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Freq"]
print(top2_df)#Barplot of most freq Bi-grams
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Bi-gram", y="Freq", data=top2_df)
h.set_xticklabels(h.get_xticklabels(), rotation=45)

In [ ]:
#Most frequently occuring Tri-grams
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range=(3,3), 
           max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in     
                  vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], 
                reverse=True)
    return words_freq[:n]
top3_words = get_top_n3_words(corpus, n=20)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Freq"]
print(top3_df)#Barplot of most freq Tri-grams
import seaborn as sns
sns.set(rc={'figure.figsize':(13,8)})
j=sns.barplot(x="Tri-gram", y="Freq", data=top3_df)
j.set_xticklabels(j.get_xticklabels(), rotation=45)

Converting to a matrix of integers

The next step of refining the word counts is using the TF-IDF vectoriser. The deficiency of a mere word count obtained from the countVectoriser is that, large counts of certain common words may dilute the impact of more context specific words in the corpus. This is overcome by the TF-IDF vectoriser which penalizes words that appear several times across the document. TF-IDF are word frequency scores that highlight words that are more important to the context rather than those that appear frequently across documents.

TF-IDF consists of 2 components:

    TF — term frequency
    IDF — Inverse document frequency
    TF=Freq of terms/total no of terms
    IDF=log(total documents)/number of documents with the term
    

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)# get feature names
feature_names=cv.get_feature_names()
 
# fetch document for which keywords needs to be extracted
doc=corpus[123]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

In [ ]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)
 
# now print the results
print("\nAbstract:")
print(doc)
print("\nKeywords:")
for k in keywords:
    print(k,keywords[k])

Ideally for the IDF calculation to be effective, it should be based on a large corpora and a good representative of the text for which the keywords need to be extracted. In our example, if we use the full article text instead of the abstracts, the IDF extraction would be much more effective. However, considering the size of the dataset, I have limited the corpora to just the abstracts for the purpose of demonstration.

This is a fairly simple approach to understand fundamental concepts of NLP and to provide a good hands-on practice with some python codes on a real-life use case. The same approach can be used to extract keywords from news feeds & social media feeds.